In this notebook, we will try to create a function to read a a parquet file (or multiple ones) and filter by IDs.

In [1]:
import pandas as pd
import numpy as np
from os.path import join
import polars as pl
import os
from glob import glob
import sys
sys.path.append('/home/Samsung2TB/')
sys.path.append('/home/Samsung2TB/astromer/')
from tqdm import tqdm
import os
from joblib import Parallel, delayed
from astromer.src.data.record import DataPipeline
import tensorflow as tf

from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
from time import time

In [2]:
local_path = '/home/Samsung2TB/Databases/MACHO/'

Medatada is now a parquet file, with the column newID, which is an integer.

In [3]:
METAPATH = join(local_path, 'metadata.parquet')
LCDIR =  join(local_path, 'parquets/')
metadata = pd.read_parquet(METAPATH)
metadata.head()

,ID,Path,N,Band,Class,newID,Chunk
0,F_103.24812.179,./R/LCs/F_103.24812.179.dat,278,R,NonVar,0,0
1,F_104.20640.4890,./R/LCs/F_104.20640.4890.dat,1603,R,NonVar,1,0
2,F_104.20000.1814,./R/LCs/F_104.20000.1814.dat,1597,R,NonVar,2,0
3,F_101.21431.87,./R/LCs/F_101.21431.87.dat,785,R,NonVar,3,0
4,F_102.22468.92,./R/LCs/F_102.22468.92.dat,414,R,NonVar,4,0


In [4]:
metadata['Class'] = pd.Categorical(metadata['Class'])
metadata['Label'] = metadata['Class'].cat.codes
metadata['Path'] = metadata['Path'].apply(lambda x: os.path.join(LCDIR, x)) 

The light curves are stored in parquet format, with the column newID to identify the object.

In [5]:
parquets = glob(LCDIR+ '*')
pl.read_parquet(parquets[0]).head()

mjd,mag,err,newID
f64,f64,f64,i64
49066.7866,-4.517,0.396,1625000
49078.797,-4.834,0.356,1625000
49082.788,-4.798,0.516,1625000
49094.7793,-4.63,0.367,1625000
49095.7186,-4.342,0.284,1625000


In each parquet file, 25000 were written. 

# Test the new functionality

First run is slower because of the caching. 

In [6]:
metadata_sample = metadata.iloc[:100].copy()

myPipeline = DataPipeline(metadata=metadata_sample, 
                            context_features=['ID', 'Label', 'Class'],
                            sequential_features=['mjd', 'mag'],)
test_metadata = metadata_sample.sample(n=10)
k_folds = 2

myPipeline.train_val_test(val_frac=0.2, 
                        test_meta=[test_metadata], 
                        k_fold=k_folds)

a = time()
var = myPipeline.run( LCDIR, METAPATH ,n_jobs=2,)
b = time()
print(b-a)

[INFO] 100 samples loaded
[INFO] Using ID col as sample identifier
[INFO] Shuffling
[INFO] Shuffling


Writting test fold 1:   0%|                               | 0/2 [00:00<?, ?it/s]

0.591048002243042


In [7]:
M=10000
metadata_sample = metadata.sample(M, replace=False)

myPipeline = DataPipeline(metadata=metadata_sample, 
                            context_features=['ID', 'Label', 'Class'],
                            sequential_features=['mjd', 'mag'],)
test_metadata = metadata_sample.sample(n=100)
k_folds = 2

myPipeline.train_val_test(val_frac=0.2, 
                        test_meta=[test_metadata], 
                        k_fold=k_folds)

[INFO] 10000 samples loaded
[INFO] Using ID col as sample identifier
[INFO] Shuffling
[INFO] Shuffling


In [8]:
a = time()
var = myPipeline.run( LCDIR, METAPATH ,n_jobs=2,)
b = time()
print(b-a)

Writting test fold 1:   0%|                               | 0/2 [00:21<?, ?it/s]

21.27983045578003
